In [1]:
import csv
import json
import struct
from functools import reduce
import numpy as np
import os

filename = 'im1_14_myatt'

## 拷贝tflite文件到flatbuffers中 并用flatc解码为json

In [ ]:
orifile = '/home/gx/myproj/generative_inpainting-master/{}.tflite'.format(filename)
disfile = '/home/gx/myproj/flatbuffers/{}.tflite'.format(filename)
os.system('cp {} {}'.format(orifile, disfile))
if os.path.isfile(disfile): 
    print('copy file success')
os.system('./flatc -t schema.fbs -- {}.tflite --strict-json'.format(filename))

copy file success


## 解析json结构

In [ ]:
jsonpath = "/home/gx/myproj/flatbuffers/{}.json".format(filename)
with open(jsonpath,'r') as f_json:
    model = json.load(f_json)

op = model['subgraphs'][0]['operators']
tensor = model['subgraphs'][0]['tensors']
opsort = model['operator_codes']
buffers = model['buffers']

## 修改reorderaxes为transpose

1. 178 transpose 赋值给185 reorderaxes
2. 给185增加一个tensor（index485）作为转置参数 内容复制index421的tensor 修改名称和buffer
3. 给index=485的新tensor增加一个buffer(index486)  data为：3000 0000 1000 2000

In [ ]:
buffers.append({'data': [3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0]})  #  index486

tensor.append({'shape': [4],                                               #  index485
               'type': 'INT32',
               'buffer': 486,
               'name': 'inpaint_net/transpose_r/perm',
               'quantization': {}})

op[185] = {'opcode_index': 12,
           'inputs': [228,485],
           'outputs': [229],
           'builtin_options_type': 'TransposeOptions',
           'builtin_options': {},
           'mutating_variable_inputs': []}

## 保存csv网络结构文件

In [ ]:
def append_row_to_csv(appendList):
    with open(filename +'.csv','a', newline = "") as f:
        csv_writer = csv.writer(f, dialect = "excel")
        csv_writer.writerow(appendList)

In [ ]:
try:
    os.remove(filename+'.csv')
except:
    pass

opsort = model['operator_codes']
op = model['subgraphs'][0]['operators']
print('op number: '+str(len(op)))
tensor = model['subgraphs'][0]['tensors']
print('tensor number: '+str(len(tensor)))
buffers = model['buffers']
print('buffers number: '+str(len(buffers)))

index = 0
for i in op: # i op字典结构
    index +=1
    temp = []
    try:
        opsortindex = i['opcode_index']
        this_op_sort = opsort[opsortindex]['builtin_code']
    except:
        this_op_sort = str(index-1)
    if this_op_sort == 'CUSTOM':
        this_op_sort = opsort[i['opcode_index']]['custom_code']
    temp.append(this_op_sort)                                      # node算子种类

    temp.append(len(i['inputs']))                                  # 输入tensor数量
    
    try:
        outnumber = len(i['outputs'])
        temp.append(outnumber)                                     # 输出tensor数量
    except:
        pass
        temp.append('x') 
        
    for j in i['inputs']:  
        temp.append(j)                                             # 输入tensor编号
        temp.append(tensor[j]['buffer'])                           # 输入tensor所属buffer编号
        if buffers[tensor[j]['buffer']]:
            temp.append('F')
        else:
            temp.append('')
        input_tensor_name = tensor[j]['name']
        temp.append(input_tensor_name)                             # 输入tensor名称
        
    for j in i['outputs']:      
        temp.append(j)                                             # 输出tensor编号
        temp.append(tensor[j]['buffer'])                           # 输出tensor所属buffer编号
        if buffers[tensor[j]['buffer']]:
            temp.append('F')
        else:
            temp.append('')
        output_tensor_name = tensor[j]['name']
        temp.append(output_tensor_name)                            # 输出tensor名称
            
    append_row_to_csv(temp)

## 保存json文件并用flatc编码为tflite文件

In [ ]:
with open(jsonpath,"w") as f:
    json.dump(model,f,indent=4)

os.system('./flatc -b schema.fbs {}.json'.format(filename))